In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common import SessionNotCreatedException,WebDriverException
import time
import pandas as pd
import multiprocessing as mp

convert_dict = {"text-red-500" : 0, "text-green-500" : 1}


NUM_WORKERS = 7
SAVE_CHUNK = 100


def parse(data,ffx):
    retries = 3
    id,row = data
    print(id)
    ffx.get(row[0] + "specs/")
    soup = BeautifulSoup(ffx.page_source,"html.parser")
    hehe = soup.find("div",attrs={"class":"inpageSections loaded","id":"section-specs"})
    better_hehe = hehe.find("div",attrs={"class":"lm-catalog-specs border-b-2 border-dashed text-lm-darkBlue border-gray-300 pt-5 pb-10"})
    next_hehe = better_hehe.find_all("ul")
    hahahaha = {}
    for data in next_hehe:
        hohoho = data.find_all("li")
        if len(hohoho)!=2:
            continue
        final_labels = [k.get_text().strip() for k in hohoho]
        if final_labels[1] == "":
            try:
                final_labels[1] = convert_dict[hohoho[1].find("i")["class"][1]]
            except TypeError:
                continue
        if "USB" in final_labels[0]:
            hehehe = final_labels[0].split()
            if len(hehehe)==3:
                hohoho = " ".join(hehehe[1:])
                hihihi = int(hehehe[0][0])
                hahahaha[hohoho] = hahahaha.get(hohoho,0) + hihihi
            elif len(hehehe)==2:
                hohoho = " ".join(hehehe)
                hahahaha[hohoho] = hahahaha.get(hohoho,0) + 1
            continue
        match final_labels[0]:
            case "CPU"|"GPU":
                hahahaha[final_labels[0]] = final_labels[1].split("#")[0]
            case "M.2 Slot":
                hahahaha[final_labels[0]] = final_labels[1].split("\n")[0]
            case _:
                if final_labels[0] in ['link','name','CPU','GPU','Display','HDD/SSD','RAM','OS','Body Material','Dimensions','Weight','M.2 Slot','USB Type-C','USB Type-A','HDMI','Bluetooth','Wi-Fi','Card Reader','Ethernet LAN','Web camera','Security Lock slot','Fingerprint reader','Backlit keyboard','Cost','Total Score','Portability Score','Display Score','Work Score','Play Score']:
                    hahahaha[final_labels[0]] = final_labels[1]
    hehe = soup.find("ul",attrs = {"class":["catalog-buttons-3 grid grid-cols-1 mt-5 gap-3 text-left","catalog-buttons-1 grid grid-cols-1 mt-5 gap-3 text-left","catalog-buttons-2 grid grid-cols-1 mt-5 gap-3 text-left"]})
    score_part = soup.find("div",attrs = {"class":"grid gap-2 grid-cols-[1fr_auto]"})
    if hehe is not None:
        hehehe = hehe.get_text("").split("$")[-1].strip()
        hahahaha["Cost"] = hehehe
    if score_part is not None:
        kirara = [line.strip() for line in score_part.get_text().splitlines() if line.strip()]
        hahahaha["Total Score"] = kirara[0]
        hahahaha["Portability Score"] = kirara[2]
        hahahaha["Display Score"] = kirara[5]
        hahahaha["Work Score"] = kirara[8]
        hahahaha["Play Score"] = kirara[11]
    ffx.quit()
    return id,hahahaha


In [2]:
import json

In [3]:
hahahaha = {"link":[],"name":[]}
ffx = webdriver.Chrome()
def hehe():
    with open("final.json","r") as f:
        mygod = json.load(f)
    first = False
    for k in range(10):
        check = False
        while check == False:
            ffx.get(f"https://laptopmedia.com/specs/?current=n_{k}_n&size=n_1000_n&sort%5B0%5D%5Bfield%5D=availability&sort%5B0%5D%5Bdirection%5D=desc&sort%5B1%5D%5Bfield%5D=date_published&sort%5B1%5D%5Bdirection%5D=desc") 
            try:
                WebDriverWait(ffx, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "flex.items-center.gap-2.list-none.border-b.py-1.mb-2"))
                )
            except:
                continue
            check = True
            soup = BeautifulSoup(ffx.page_source,"html.parser")
            hehe = soup.find_all("li",attrs={"class":"flex items-center gap-2 list-none border-b py-1 mb-2"})
            for id,target in enumerate(hehe):
                hahahaha["link"].append(target.find("a")["href"])
                hahahaha["name"].append(target.find("img")["alt"])
                if not first:
                    first = target.find("a")["href"]
                if target.find("a")["href"]==mygod:
                    with open("final.json","w") as f:
                        json.dump(first,f)
                    return
                if id>999:
                    break
                
hehe()

In [4]:
myhahaha = pd.DataFrame(hahahaha)

args = [(id,row) for (id,row) in myhahaha.iterrows()]
ffx = webdriver.Chrome()
for arg in args:
    id,hahahahahaha = parse(arg,ffx)
    for key, value in hahahahahaha.items():
        myhahaha.at[id,key] = value
    myhahaha.to_csv("laptop_2.csv", index=False)

0
